In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    probs = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=probs, replace=False)
    
    return recs.tolist()

In [6]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['sales_log'] = items_weights['sales_value'].apply(lambda x: np.log(x+10e5))
total_amount = items_weights['sales_log'].sum()
items_weights['weight'] = items_weights['sales_log'].apply(lambda x: x / total_amount)
items_weights = items_weights.drop(['sales_value', 'sales_log'], axis = 1)
# items_weights
result['weighted_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n = 5))

Wall time: 3.55 s


In [7]:
result.head(5)

,user_id,actual,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[46119, 945997, 15596765, 2189289, 2462772]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8205495, 10455959, 953539, 833441, 13007531]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1059342, 1885849, 1004173, 1816907, 13189849]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [8]:
from metrics import precision_at_k, recall_at_k

In [9]:
res = pd.read_csv('predictions_basic.csv')
res.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [10]:
res['actual'] = res['actual'].replace(regex = True, to_replace = r'\n', value = '').replace(regex = True, to_replace = r' +', value = ', ').replace(regex = True, to_replace = r'^\[, ', value = '[')
res.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [11]:
metrics = {}
for method in res.columns[2:]:
    metrics[method] = res.apply(lambda row: precision_at_k(eval(row[method]), eval(row['actual']), 5), axis=1).mean()
metrics

{'random_recommendation': 0.0005876591576885406,
 'popular_recommendation': 0.15523996082272082,
 'itemitem': 0.033594515181195064,
 'cosine': 0.03525954946131262,
 'tfidf': 0.03614103819784544,
 'own_purchases': 0.1799869409076044}

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Посчитаем бейзлайны на всех товарах

In [12]:
def random_recommendation(items, n=5, exclude = []):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    ind = np.intersect1d(items, exclude, return_indices=True)[2]

    items = np.delete(items, ind)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 17.5 s


,user_id,actual,weighted_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]"


In [14]:
def popularity_recommendation(data, n=5, exclude = []):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    exclude = np.array(exclude)
    popular = popular.loc[~popular['item_id'].isin(exclude)]
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 119 ms


,user_id,actual,weighted_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [16]:
# base_metrics = {}
# for method in result.columns[2:]:
#     base_metrics[method] = result.apply(lambda row: precision_at_k(row[method], row['actual'], 5), axis=1).mean()
# base_metrics

Отберем топ-5000 товаров

In [17]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [18]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [19]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)


data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 ## точно покупал или все-таки НЕ покупал??

C:\Users\snetkova\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
%%time
exclude = [999999]
#weighted top5000
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights = items_weights.loc[~items_weights['item_id'].isin(np.array(exclude))]

items_weights['sales_log'] = items_weights['sales_value'].apply(lambda x: np.log(x+10e5))
total_amount = items_weights['sales_log'].sum()
items_weights['weight'] = items_weights['sales_log'].apply(lambda x: x / total_amount)
items_weights = items_weights.drop(['sales_value', 'sales_log'], axis = 1)


result['weighted_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n = 5))
result.head(5)

Wall time: 326 ms


,user_id,actual,weighted_recommendation,random_recommendation,popular_recommendation,weighted_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1068957, 12132685, 911622, 1037965, 1021324]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534657, 964968, 1105034, 845208, 868888]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[46119, 945997, 15596765, 2189289, 2462772]","[12582321, 104681, 1090840, 2197479, 1491193]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5563934, 1059891, 1079228, 1096036, 7466806]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8205495, 10455959, 953539, 833441, 13007531]","[916122, 1039028, 9555387, 235818, 12171944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5565202, 866878, 885023, 9420286, 1045109]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1059342, 1885849, 1004173, 1816907, 13189849]","[1514813, 1097237, 10149451, 12387393, 862035]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1055503, 1046262, 919596, 6533936, 1053924]"


In [21]:
%%time

items = data_train.item_id.unique()


result['random_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5, exclude =[999999]))
result.head(2)


Wall time: 952 ms


,user_id,actual,weighted_recommendation,random_recommendation,popular_recommendation,weighted_top5000,random_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1068957, 12132685, 911622, 1037965, 1021324]","[894425, 1122405, 864279, 919704, 1076769]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534657, 964968, 1105034, 845208, 868888]","[1060529, 995055, 990804, 1035676, 982984]"


In [22]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5, exclude = [999999])

result['popular_top5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 61.9 ms


,user_id,actual,weighted_recommendation,random_recommendation,popular_recommendation,weighted_top5000,random_top5000,popular_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1068957, 12132685, 911622, 1037965, 1021324]","[894425, 1122405, 864279, 919704, 1076769]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534657, 964968, 1105034, 845208, 868888]","[1060529, 995055, 990804, 1035676, 982984]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [23]:
metrics = {}
for method in result.columns[2:]:
    metrics[method] = result.apply(lambda row: precision_at_k(row[method], row['actual'], 5), axis=1).mean()
metrics

{'weighted_recommendation': 0.0007835455435847208,
 'random_recommendation': 0.00039177277179236047,
 'popular_recommendation': 0.15523996082272082,
 'weighted_top5000': 0.0062683643486777605,
 'random_top5000': 0.00617042115572967,
 'popular_top5000': 0.15523996082272082}

Вполне логично, что метрики на популярных рекомендациях не изменились, ткк самые популярные 5 из всех и самые популярные 5 из топ-5000 - одни и те же товары. Для остальных бейзлайнов метрики немного улучшились

Посмотрим на ItemItemRecommender с разным числом соседей

In [24]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 ) #остались только из топ-5000 и 999999 вместо остальных

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать (количество -> да/нет)
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [25]:
user_item_matrix.iloc[:, 1:].sum().describe()

count    5000.00000
mean      133.41560
std       136.68634
min         1.00000
25%        68.00000
50%        97.00000
75%       146.00000
max      2498.00000
dtype: float64

In [26]:
user_item_matrix.iloc[:, 1:].sum()[999999] # очевидно, что фейковый 999999 оказался самым популярным товаром

2498.0

In [27]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

In [28]:
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

In [29]:
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

In [30]:
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [31]:
for neighbour in [1, 2, 3, 5, 8, 13, 21]:
    model = ItemItemRecommender(K=neighbour, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    result['itemitem_'+ str(neighbour)] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]], #убрала 999999
                                        recalculate_user=True)])

In [32]:
result.head()

,user_id,actual,weighted_recommendation,random_recommendation,popular_recommendation,weighted_top5000,random_top5000,popular_top5000,itemitem_1,itemitem_2,itemitem_3,itemitem_5,itemitem_8,itemitem_13,itemitem_21
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423723, 6533126, 1759896, 963024, 1299065]","[1117382, 905999, 893583, 10456208, 1355578]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1068957, 12132685, 911622, 1037965, 1021324]","[894425, 1122405, 864279, 919704, 1076769]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 1127831, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[982704, 13115694, 12300976, 2907438, 1097373]","[9878736, 1035275, 901464, 5556558, 5564206]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534657, 964968, 1105034, 845208, 868888]","[1060529, 995055, 990804, 1035676, 982984]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[46119, 945997, 15596765, 2189289, 2462772]","[12582321, 104681, 1090840, 2197479, 1491193]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5563934, 1059891, 1079228, 1096036, 7466806]","[1043128, 1064213, 1078011, 1023815, 1100555]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 1029743, 6534178, 1127831, 995785]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 1098066, 840361]","[1082185, 981760, 840361, 995242, 1127831]","[1082185, 981760, 995242, 840361, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8205495, 10455959, 953539, 833441, 13007531]","[916122, 1039028, 9555387, 235818, 12171944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5565202, 866878, 885023, 9420286, 1045109]","[878445, 1070169, 9859111, 943233, 950384]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 1029743, 1127831, 995785, 1044078]","[1082185, 995242, 1029743, 826249, 1127831]","[1082185, 981760, 995242, 1029743, 826249]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 995242, 840361, 1098066]","[1082185, 981760, 995242, 840361, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1059342, 1885849, 1004173, 1816907, 13189849]","[1514813, 1097237, 10149451, 12387393, 862035]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1055503, 1046262, 919596, 6533936, 1053924]","[995965, 1005921, 1055072, 1060673, 6548453]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 1029743, 1098066, 6534178, 1127831]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 6534178, 995242, 1029743]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 995242, 840361, 1098066]","[1082185, 981760, 1098066, 995242, 840361]"


In [33]:
metrics = {}
for method in result.columns[2:]:
    metrics[method] = result.apply(lambda row: precision_at_k(row[method], row['actual'], 5), axis=1).mean()
metrics

C:\Users\snetkova\AI_rec_sys\les02\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


{'weighted_recommendation': 0.0007835455435847208,
 'random_recommendation': 0.00039177277179236047,
 'popular_recommendation': 0.15523996082272082,
 'weighted_top5000': 0.0062683643486777605,
 'random_top5000': 0.00617042115572967,
 'popular_top5000': 0.15523996082272082,
 'itemitem_1': 0.20191740412979084,
 'itemitem_2': 0.2193437806072451,
 'itemitem_3': 0.21897649363368973,
 'itemitem_5': 0.15122428991184922,
 'itemitem_8': 0.1646425073457374,
 'itemitem_13': 0.16640548481880305,
 'itemitem_21': 0.16679725759059535}

В нашем случае наилучший результат достигнут при двух соседях